In [1]:
cd("/media/u01/analytics/scoring/k/Santander/")
using DataFrames, XGBoost

In [2]:
traincsv = "train.csv"
testcsv = "test.csv"
samplesubmission = "sample_submission.csv"
# read data
df = readtable(traincsv)
dft = readtable(testcsv)
train_X = array(df[:,2:370])
train_Y = array(df[:TARGET])
test_X = array(dft[:,2:370])
print("Training Size=", size(train_X), "\n")
print("Testing Size=", size(test_X), "\n")

 in depwarn at deprecated.jl:73
 in array at deprecated.jl:50
 in include_string at loading.jl:266
 in execute_request_0x535c5df2 at /home/rmadmin/.julia/v0.4/IJulia/src/execute_request.jl:183
 in eventloop at /home/rmadmin/.julia/v0.4/IJulia/src/IJulia.jl:143
 in anonymous at task.jl:447
while loading In[2], in expression starting on line 7
 in depwarn at deprecated.jl:73
 in array at deprecated.jl:50
 in include_string at loading.jl:266
 in execute_request_0x535c5df2 at /home/rmadmin/.julia/v0.4/IJulia/src/execute_request.jl:183
 in eventloop at /home/rmadmin/.julia/v0.4/IJulia/src/IJulia.jl:143
 in anonymous at task.jl:447
while loading In[2], in expression starting on line 9


Training Size=(76020,369)
Testing Size=(75818,369)


In [3]:
# train xgboost
num_round = 100
bst = xgboost(train_X, num_round, label=train_Y, eta=1, max_depth=2)
# predict on testing data
pred = XGBoost.predict(bst, test_X)
# clip prediction value between [0.0, 1.0]
pred = [min(max(pred[i], 0.0), 1.0) for i = 1:length(pred)]
print("Prediction Size=", size(pred), "\n")
print("Mean Prediction=", mean(pred), "\n")
print("Max Prediction=", minimum(pred), "\n")
print("Min Prediction=", maximum(pred), "\n")

[1]	train-rmse:0.188873
[2]	train-rmse:0.188376
[3]	train-rmse:0.187878
[4]	train-rmse:0.187549
[5]	train-rmse:0.187271
[6]	train-rmse:0.187063
[7]	train-rmse:0.186923
[8]	train-rmse:0.186584
[9]	train-rmse:0.186403
[10]	train-rmse:0.186317
[11]	train-rmse:0.186257
[12]	train-rmse:0.186206
[13]	train-rmse:0.186106
[14]	train-rmse:0.186044
[15]	train-rmse:0.185959
[16]	train-rmse:0.185855
[17]	train-rmse:0.185801
[18]	train-rmse:0.185717
[19]	train-rmse:0.185638
[20]	train-rmse:0.185577
[21]	train-rmse:0.185501
[22]	train-rmse:0.185468
[23]	train-rmse:0.185357
[24]	train-rmse:0.185316
[25]	train-rmse:0.185255
[26]	train-rmse:0.185214
[27]	train-rmse:0.185156
[28]	train-rmse:0.185098
[29]	train-rmse:0.185060
[30]	train-rmse:0.185013
[31]	train-rmse:0.184968
[32]	train-rmse:0.184914
[33]	train-rmse:0.184837
[34]	train-rmse:0.184797
[35]	train-rmse:0.184768
[36]	train-rmse:0.184741
[37]	train-rmse:0.184704
[38]	train-rmse:0.184658
[39]	train-rmse:0.184633
[40]	train-rmse:0.184603
[41]	trai

Prediction Size=(75818,)
Mean Prediction=0.04166316875472078
Max Prediction=0.0
Min Prediction=1.0


In [ ]:
splsub = readtable(samplesubmission)
splsub[:,2] = pred
writetable("submission.csv", splsub)

In [ ]:
#################### OR ToDo - Conert from R#####################################

In [ ]:
library(xgboost)
library(Matrix)
set.seed(1234)
train <- read.csv("train.csv")
test  <- read.csv("test.csv")
##### Removing IDs
train$ID <- NULL
test.id <- test$ID
test$ID <- NULL
##### Extracting TARGET
train.y <- train$TARGET
train$TARGET <- NULL
##### 0 count per line
count0 <- function(x) {
    return( sum(x == 0) )
}
train$n0 <- apply(train, 1, FUN=count0)
test$n0 <- apply(test, 1, FUN=count0)

##### Removing constant features
cat("\n## Removing the constants features.\n")
for (f in names(train)) {
    if (length(unique(train[[f]])) == 1) {
        cat(f, "is constant in train. We delete it.\n")
        train[[f]] <- NULL
        test[[f]] <- NULL
    }
}

##### Removing identical features
features_pair <- combn(names(train), 2, simplify = F)
toRemove <- c()
for(pair in features_pair) {
    f1 <- pair[1]
    f2 <- pair[2]
    
    if (!(f1 %in% toRemove) & !(f2 %in% toRemove)) {
        if (all(train[[f1]] == train[[f2]])) {
            cat(f1, "and", f2, "are equals.\n")
            toRemove <- c(toRemove, f2)
        }
    }
}

feature.names <- setdiff(names(train), toRemove)

train <- train[, feature.names]
test <- test[, feature.names]

train$TARGET <- train.y


train <- sparse.model.matrix(TARGET ~ ., data = train)

dtrain <- xgb.DMatrix(data=train, label=train.y)
watchlist <- list(train=dtrain)

param <- list(  objective           = "binary:logistic", 
                booster             = "gbtree",
                eval_metric         = "auc",
                eta                 = 0.02,
                max_depth           = 5,
                subsample           = 0.7,
                colsample_bytree    = 0.7
)

clf <- xgb.train(   params              = param, 
                    data                = dtrain, 
                    nrounds             = 560, 
                    verbose             = 2,
                    watchlist           = watchlist,
                    maximize            = FALSE
)


test$TARGET <- -1
test <- sparse.model.matrix(TARGET ~ ., data = test)

preds <- predict(clf, test)
submission <- data.frame(ID=test.id, TARGET=preds)
cat("saving the submission file\n")
write.csv(submission, "submission.csv", row.names = F)